In [7]:
'''
Project:    Portfolio Risk Analysis
Objective:  Complete all the steps below and return the risk analysis for seven stock
            porfolio against ETFs
Author:     Jose Aponte
Class:      ECO 41552 - Quantitative Finance
'''

'\nProject:    Portfolio Risk Analysis\nObjective:  Complete all the steps below and return the risk analysis for seven stock\n            porfolio against ETFs\nAuthor:     Jose Aponte\nClass:      ECO 41552 - Quantitative Finance\n'

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
#################### Tickers ####################
"""""
These stocks were chosen by weight from S&P500
AAPL =  Apple Inc
MSFT =  Microsoft Corporation
AMAZN = Amazon.com Inc
TSLA =  Tesla Inc
GOOGL = Alphabet Inc. Class A
UNH =   UnitedHealth Group Incorporate
JNJ =   Johnson and Johnson
"""""
tickers = ['AAPL','MSFT','AMZN','TSLA','GOOGL','UNH','JNJ']
#################### Exchange Traded Fund ####################
"""
SPY =   SPDR S&P 500 ETF Trust
IWM =   iShares Russell 2000 ETF
DIA =   SPDR Dow Jones Industrial Average ETF Trust
"""
etf = ['SPY','IWM','DIA']
dataFinance = yf.download(tickers+etf, period= ' 10y')
dataFinance = dataFinance['Adj Close']
dataFinance.head()


#################### Portfolio Weight Calculator ####################
dataFinance['Portfolio'] = dataFinance[tickers].mean(axis=1)
returns = dataFinance.pct_change()
table = pd.DataFrame(index=tickers)
table['Portfolio Weight [%]'] = (format((1/len(tickers))*100,'.2f'))

########## Annualized Volatility ##########
# Number 63 means 63-days ~ 3-month back until today 
volatilityTrailing3Month = returns[-63:].dropna()
table['Annualized Volatility (Trailing 3-months)'] = (((volatilityTrailing3Month.var()/volatilityTrailing3Month.std())**(1/np.sqrt(4)))*100)

#################### Beta against ETF ####################
# Number 252 means 252-days ~ 12-month back until today
beta = returns[-252:].cov()/returns[-252:].var()
for eachItem in etf:
    table['Beta Against '+eachItem]=beta[eachItem]

#################### Drawdown Calculation ####################
drawdown = (returns[-252:].rolling(5).min()-returns[-252:].rolling(5).max())/returns[-252:].rolling(5).max()
table['Average Weekly Drawdown'] = drawdown.mean()
table['Maximum Weekly Drawdown'] = drawdown.max()

######## Total Return and Annualized Total Return (using trailing 10-years) ###########

table['TotalReturn']=dataFinance.pct_change(len(dataFinance)-1)[-1:].T
table['Annualized Total Returns']=table.TotalReturn**(1/np.sqrt(252))

table.T


[*********************100%***********************]  10 of 10 completed


,AAPL,MSFT,AMZN,TSLA,GOOGL,UNH,JNJ
Portfolio Weight [%],14.29,14.29,14.29,14.29,14.29,14.29,14.29
Annualized Volatility (Trailing 3-months),13.982169,13.922798,16.698251,17.996776,15.078384,11.905875,10.272818
Beta Against SPY,1.22972,1.204965,1.610286,1.823977,1.30536,0.642829,0.291
Beta Against IWM,0.881728,0.835079,1.199595,1.522787,0.917186,0.398439,0.122997
Beta Against DIA,1.363636,1.308703,1.726984,1.79599,1.412363,0.84902,0.440782
Average Weekly Drawdown,-2.094912,-2.695035,-1.328757,-3.505494,-3.227105,-3.865046,-2.101873
Maximum Weekly Drawdown,21.666143,39.152941,415.16831,6.799648,14.960796,3.289699,88.421339
TotalReturn,6.441193,9.104298,8.287424,121.117948,4.347755,9.520021,2.068386
Annualized Total Returns,1.124502,1.149283,1.142497,1.352788,1.097001,1.15252,1.046846
